# Parsing Coronavirus Data

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)

## new york times

#### Grab raw data from nytimes website

In [2]:
url = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
df = pd.read_csv(url, index_col="date", parse_dates=True)

In [3]:
df.head()  # table for all counties, first 5 entries

,county,state,fips,cases,deaths
date,,,,,
2020-01-21,Snohomish,Washington,53061.0,1,0
2020-01-22,Snohomish,Washington,53061.0,1,0
2020-01-23,Snohomish,Washington,53061.0,1,0
2020-01-24,Cook,Illinois,17031.0,1,0
2020-01-24,Snohomish,Washington,53061.0,1,0


#### Pull only King County's data

In [4]:
kc = df.query("county == 'King'").copy()
kc.sort_values(by="date", inplace=True)
kc.head()

,county,state,fips,cases,deaths
date,,,,,
2020-02-28,King,Washington,53033.0,1,0
2020-02-29,King,Washington,53033.0,4,1
2020-03-01,King,Washington,53033.0,11,3
2020-03-02,King,Washington,53033.0,15,6
2020-03-03,King,Washington,53033.0,22,10


In [5]:
kc

,county,state,fips,cases,deaths
date,,,,,
2020-02-28,King,Washington,53033.0,1,0
2020-02-29,King,Washington,53033.0,4,1
2020-03-01,King,Washington,53033.0,11,3
2020-03-02,King,Washington,53033.0,15,6
2020-03-03,King,Washington,53033.0,22,10
2020-03-04,King,Washington,53033.0,33,11
2020-03-05,King,Washington,53033.0,52,11
2020-03-06,King,Washington,53033.0,59,12
2020-03-07,King,Washington,53033.0,72,16


#### Save to csv

In [6]:
# change name inside parenthesis to the filename
# if you want the csv for all counties, it's under the variable name "df" instead of "kc"
kc.to_csv("king_county.csv")

## coronadatascraper

In [15]:
# from https://coronadatascraper.com/
# data here: https://coronadatascraper.com/timeseries-tidy.csv.zip
cds = pd.read_csv("timeseries-tidy.csv", index_col='date', parse_dates=True)
cds.query("(county == 'King County') & (state == 'Washington') & (type == 'cases')").sort_values(by='date')

/Users/alex/.pyenv/versions/38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,name,level,city,county,state,country,population,lat,long,aggregate,tz,type,value
date,,,,,,,,,,,,,
2020-01-22,"King County, Washington, United States",county,NaN,King County,Washington,United States,2252782.0,47.433,-121.797,county,America/Los_Angeles,cases,1.0
2020-01-23,"King County, Washington, United States",county,NaN,King County,Washington,United States,2252782.0,47.433,-121.797,county,America/Los_Angeles,cases,1.0
2020-01-24,"King County, Washington, United States",county,NaN,King County,Washington,United States,2252782.0,47.433,-121.797,county,America/Los_Angeles,cases,1.0
2020-01-25,"King County, Washington, United States",county,NaN,King County,Washington,United States,2252782.0,47.433,-121.797,county,America/Los_Angeles,cases,1.0
2020-01-26,"King County, Washington, United States",county,NaN,King County,Washington,United States,2252782.0,47.433,-121.797,county,America/Los_Angeles,cases,1.0
2020-01-27,"King County, Washington, United States",county,NaN,King County,Washington,United States,2252782.0,47.433,-121.797,county,America/Los_Angeles,cases,1.0
2020-01-28,"King County, Washington, United States",county,NaN,King County,Washington,United States,2252782.0,47.433,-121.797,county,America/Los_Angeles,cases,1.0
2020-01-29,"King County, Washington, United States",county,NaN,King County,Washington,United States,2252782.0,47.433,-121.797,county,America/Los_Angeles,cases,1.0
2020-01-30,"King County, Washington, United States",county,NaN,King County,Washington,United States,2252782.0,47.433,-121.797,county,America/Los_Angeles,cases,1.0


## hopkins

In [ ]:
# data here: https://github.com/CSSEGISandData/COVID-19


## usafacts

In [17]:
ufacts = "covid_confirmed_usafacts.csv"
fcts = pd.read_csv(ufacts)

In [18]:
fcts

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,74,84,91,93,103,103,110,110,120,127
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,222,224,227,231,243,244,254,254,260,262
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,59,61,67,69,74,79,79,81,85,90
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,46,46,46,46,46,49,50,50,50,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,56037,Sweetwater County,WY,56,0,0,0,0,0,0,...,12,13,13,13,14,15,16,16,16,24
3191,56039,Teton County,WY,56,0,0,0,0,0,0,...,67,67,67,68,68,68,69,69,69,100
3192,56041,Uinta County,WY,56,0,0,0,0,0,0,...,7,7,7,7,7,7,8,8,8,11
3193,56043,Washakie County,WY,56,0,0,0,0,0,0,...,5,5,5,5,5,5,6,13,13,16


In [21]:
kc = fcts.query("countyFIPS == 53033")
kc

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20
3019,53033,King County,WA,53,1,1,1,1,1,1,...,7007,7068,7137,7212,7290,7325,7426,7472,7515,7565


In [23]:
kc.melt(id_vars = ["countyFIPS", "County Name", "State", "stateFIPS"], var_name="date")

,countyFIPS,County Name,State,stateFIPS,date,value
0,53033,King County,WA,53,1/22/20,1
1,53033,King County,WA,53,1/23/20,1
2,53033,King County,WA,53,1/24/20,1
3,53033,King County,WA,53,1/25/20,1
4,53033,King County,WA,53,1/26/20,1
5,53033,King County,WA,53,1/27/20,1
6,53033,King County,WA,53,1/28/20,1
7,53033,King County,WA,53,1/29/20,1
8,53033,King County,WA,53,1/30/20,1
9,53033,King County,WA,53,1/31/20,1
